In [12]:
from bing_image_downloader import downloader

search_term = "stripes black and white"
downloader.download(search_term, limit=500, output_dir="data/bid")

[%] Downloading Images to /home/paul/PycharmProjects/trustworthy_machine_learning/exercise_4/data/bid/stripes black and white


[!!]Indexing page: 1

[%] Indexed 150 Images on Page 1.


[%] Downloading Image #1 from http://3.bp.blogspot.com/-9UgqX6oZEjI/Tx-PgQUkn3I/AAAAAAAADTw/_iJDnTbyQKU/s1600/free%2Bdigital%2Bscrapbook%2Bpaper_black%2Band%2Bwhite%2Bstripes.jpg
[%] File Downloaded !

[%] Downloading Image #2 from http://clipground.com/images/black-and-white-striped-clipart-1.jpg
[%] File Downloaded !

[%] Downloading Image #3 from http://sweetclipart.com/multisite/sweetclipart/files/pattern_lines_black_white.png
[%] File Downloaded !

[%] Downloading Image #4 from https://paperpackagingplace.com/wp-content/uploads/2017/07/020.jpg
[%] File Downloaded !

[%] Downloading Image #5 from http://1.bp.blogspot.com/-k46HWPFRHhw/Tx-Pgm5BBBI/AAAAAAAADT8/NpXLJJz2M1Q/s1600/free%2Bdigital%2Bscrapbook%2Bpaper_black%2Band%2Bcream%2Bstripes.jpg
[%] File Downloaded !

[%] Downloading Image #6 from http

In [ ]:
from bing_image_downloader import downloader

search_term = "zebras"
downloader.download(search_term, limit=100, output_dir="data/zebras")

In [1]:
# load the caltech dataset
import numpy as np
import torch
import torchvision
from torchvision.datasets import ImageFolder
from sklearn.model_selection import train_test_split

folder = "data"

crop = [224, 224]
resize = [256, 256]
mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]

transoformation = torchvision.transforms.Compose([
    torchvision.transforms.Resize(resize),
    torchvision.transforms.CenterCrop(crop),
    torchvision.transforms.ToTensor()
])

norm = torchvision.transforms.Normalize(mean, std)
to_image = torchvision.transforms.ToPILImage()

def pick_data(dataset, size):
    _, test_ind = train_test_split(
        list(range(len(dataset.targets))), 
        test_size=size, 
        stratify=dataset.targets, 
        shuffle=True
    )
    return torch.utils.data.Subset(dataset, test_ind)

# split of data because len(d1) = 9145 is way to big lets pick like 400 examples
amount = 75

d1 = ImageFolder(f"{folder}/Caltech101/", transform=transoformation)
d1.targets = np.ones(len(d1))
d1 = pick_data(d1, amount)

d2 = ImageFolder(f"{folder}/bid/", transform=transoformation)
d2 = pick_data(d2, amount)

# Concat both datasets
dataset = torch.utils.data.ConcatDataset([d1, d2])

dataloader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=True)

### Some remarks about resnets
- So the idea of resnets is to learn the function f of an ann as residual functions  
$f(x) = f_1 \circ f_2, ... \circ f_n$ where $f_i$ is consits of ann layers aswell
- the number in resnet\<number\> is the number of convolutional layers inside the network
- For this task we should take the panultimate resnet layer as the splitting point. 

### Idea: 
- Split resnet into f(x) = h and g(h) = y and calculate the following:
    - h, y and $\frac{\partial zebra}{\partial h(y)}$ 
    - train classifier for h and the labels
    - calc tcav

### Reality: 
- Pauls adventure with strange bugs(?), memory leaks and incompetence

In [19]:
# load and decompose resnet 50
from torch import nn
import torchvision

resnet = torchvision.models.resnet50(pretrained = True)

act_dict = {}
output_dict = {}
grad_dict = {}

def hook(_, __, output):
    act_dict[ind] = (output, label)

def bw_hook(_, grad_input, __):
    grad_dict[ind] = grad_input

resnet.layer3.register_backward_hook(bw_hook)
resnet.layer3.register_forward_hook(hook)

resnet.requires_grad_(True)
########################### RANT ###########################
# FOR SOME FUCKING REASON THERE EXISTS A MEMORY LEAK AND 
# THIS CAUSES THE PROGRAMM TO CRASH AFTER N ITERATIONS 
# (EVEN IN COLAB)
########################### RANT ###########################
di = iter(dataloader)
j = 0

zebra = torch.zeros(1000)
zebra[341] = 1

In [20]:
import pickle
# So to avoid the memory leak to appear we just cant iterate through
# the complete dataset, we save our results to the drive just because
# it is not really neccessary in the current state of the code but whatever... 

for _ in range(15):
    for ind in range(10):
        j += 1
        batch, label = next(di)
        o = resnet(batch)
        output_dict[ind] = o
        loss = (o - zebra).sum()
        loss.backward()
            
    with open(f"data/pickle_files/forward/{j}.pkl", 'wb') as f:
                pickle.dump(act_dict, f)

    with open(f"data/pickle_files/output/{j}.pkl", 'wb') as f:
                pickle.dump(output_dict, f)

    with open(f"data/pickle_files/grad/{j}.pkl", 'wb') as f:
                pickle.dump(grad_dict, f)

In [21]:
# YAY now we have all the stuff we need to calculate the tcav score,
# we just need to load it from the drive
import glob
forward_pickles = glob.glob("data/pickle_files/forward/*.pkl")
output_pickles = glob.glob("data/pickle_files/output/*.pkl")
grad_pickles = glob.glob("data/pickle_files/grad/*.pkl")

def load_dict(files):
    o_dict = {}
    for i, fp in enumerate(files):
        with open(fp, "rb") as f:
            ld:dict[int, tuple[any, any]] = pickle.load(f)
            for k, v in ld.items():
                o_dict[i * 10 + k] = v
    return o_dict

act_dict = load_dict(forward_pickles)
output_dict = load_dict(output_pickles)
grad_dict = load_dict(grad_pickles)



In [22]:
# Transform the loaded data into Tensors
activations = torch.Tensor()
outputs = torch.Tensor()
grads = torch.Tensor()
labels = []

for v in output_dict.values():
    outputs = torch.cat((outputs, v))

for i, (o, l) in act_dict.items():
    activations = torch.cat((activations, o))
    if l > 0:
        l = 1
    else:
        l = 0
    labels.append(l)

for k, (v, ) in grad_dict.items():
    grads = torch.cat((grads, v))

labels = torch.Tensor(labels)

In [23]:
print(labels.shape)
print(activations.shape)
print(grads.shape)

torch.Size([150])
torch.Size([150, 1024, 14, 14])
torch.Size([150, 1024, 14, 14])


In [24]:
from sklearn.linear_model import LogisticRegression
clf_input = torch.flatten(activations, start_dim=1)
clf = LogisticRegression(random_state=0, max_iter=1000).fit(clf_input.detach().numpy(), labels.detach().numpy())

In [44]:
# 
cav = torch.Tensor(clf.coef_)
grad_f = torch.flatten(grads, start_dim=1)

concept_sens = torch.matmul(grad_f, cav.T)

pos_concept = concept_sens[concept_sens >= 0]
print(f"If i did everything somehow correctly the TCAV score should be: {pos_concept.shape[0] / concept_sens.shape[0]:.2f}")

If i did everything somehow correctly the TCAV score should be: 0.61
